In this hands-on we will use the `smolagents` library developed by HuggingFace. We use this library due to its simplicity, support for any LLM posted on HuggingFace Hub and its integration of Code Agents (more on that later).

<img src="https://camo.githubusercontent.com/c6efa99360afde7cf829dff3cad81e56573658c1843464dff1fbb30a8f63b082/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f68756767696e67666163652f646f63756d656e746174696f6e2d696d616765732f7265736f6c76652f6d61696e2f736d6f6c6167656e74732f736d6f6c6167656e74732e706e67" alt="drawing" width="400"/>

[Smolagents Documentation](https://huggingface.co/docs/smolagents/en/index)

In this hands-on we will:
- Understand why it's helpful to have agentic capabilities
- Understand how to use the `smolagents` library
- Understand the difference between a Tool Calling Agent and a Code Agent
- Implement a custom Agent leveraging the RAG pipeline that we implemented before

## 1. Setting up the environment

We will use `gpt-4.1-mini` as our LLM for this hands-on. We could also use any model available on HuggingFace. 

In [1]:
import sys
sys.path.append("../../")

In [2]:
!pip install -r ../../requirements.txt

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [5]:
from smolagents import LiteLLMModel
from IPython.display import display_markdown

model = LiteLLMModel(model_id="gpt-4.1-mini")

In [6]:
def compute_agent_cost(agent, model_name="gpt-4.1-mini"):
    if model_name == "gpt-4o-mini":
        input_token_price = 0.15 / 1000000
        output_token_price = 0.6 / 1000000
    elif model_name == "gpt-4o":
        input_token_price = 2.5 / 1000000
        output_token_price = 10 / 1000000        
    elif model_name == "gpt-4.1-mini":
        input_token_price = 0.40 / 1000000
        output_token_price = 1.6 / 1000000 
    return (
        input_token_price * agent.monitor.get_total_token_counts()["input"]
        + output_token_price * agent.monitor.get_total_token_counts()["output"]
    )

## 2. Let's create our first Agent

## 2.1 Tool Calling Agent

In [7]:
from smolagents import ToolCallingAgent

# This is as simple as
agent = ToolCallingAgent(
    tools=[],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)

In [8]:
output = agent.run("What can you do?")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I can assist with a variety of tasks including answering questions, performing calculations, providing explanations, generating text based on prompts, summarizing information, and more. I have access to tools that allow me to provide final answers to problems efficiently. Please let me know what specific task or question you have, and I'll do my best to help!

Cost of the agent:  0.00046600000000000005


Ok, so our agent says that it can help us answering questions. Let's see how it goes.

In [9]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are recent news?"
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I cannot directly visit websites or retrieve live data from them. However, I suggest you visit https://www.swissinfo.ch/eng/ directly to view the most recent news updates.

Cost of the agent:  0.0004244


In [10]:
agent.visualize()

ToolCallingAgent | gpt-4.1-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [11]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you

I guess we forgot to give tools to our agent. Let's add a websearch tool. We can use the one provided by default by the smollagents library.

In [12]:
from smolagents import VisitWebpageTool

visit_webpage_tool = VisitWebpageTool()
agent = ToolCallingAgent(
    tools=[visit_webpage_tool],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)
agent.visualize()

ToolCallingAgent | gpt-4.1-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name          ┃ Description                                  ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ visit_webpage │ Visits a webpage at the given url and reads  │ url (`string`): The url of the webpage to    │
    │               │ its content as a markdown string. Use this   │ visit.                                       │
    │               │ to browse webpages.                          │                                              │
    │ final_answer  │ Provides a final answer to the given         │ answer (`any`): The final answer to the      │
    │               │ problem.                                     │ problem                                      │
    └───────────────┴──────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [13]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are recent news?"
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

Recent news on https://www.swissinfo.ch/eng/ includes: 
- Switzerland approves clean reserve electricity plants (May 15, 2025)
- Rare blue diamond sold for $21.5 million in Geneva (May 15, 2025)
- Switzerland bans aluminium imports from Russia and Belarus (May 15, 2025)
- Immigration limits included in Swiss-EU deal (May 14, 2025)
- Switzerland outlines EU electricity deal for consumers (May 14, 2025)
- Pharma tariff threat tests Roche US investment ambitions (May 14, 2025)
- Swiss government backs down to industry outcry over energy security plans (May 14, 2025)
- Secure AI system recommended for Swiss parliament (May 14, 2025)
- Switzerland sets aside more funds to protect religious minorities (May 14, 2025)
- Swiss government ministers travelled more in 2024 (May 14, 2025)
- Suspect bombers with Russian links arrested in Switzerland (May 14, 2025)
- Proposed Swiss surveillance law ‘identical to Russia’ (May 14, 2025)
- Bonuses cuts of ex-Credit Suisse bosses ruled unlawful (May 14, 2025)
- Swiss pension funds had a good year in 2024 (May 13, 2025)
- Swiss increasingly question gender roles (May 13, 2025)
- Swiss researchers develop living material from fungi (May 13, 2025)
- UBS expects slow Swiss growth due to Trump (May 13, 2025)
- Gotthard Pass to re-open on Friday (May 13, 2025)
- Work atmosphere more important than salary, say Swiss (May 13, 2025)
- Organic farming stagnates in Switzerland (May 13, 2025)
- Who will come to the aid of a humanitarian system left behind by the West? (May 14, 2025)
- Humanitarian community sounds the alarm over famine risks in Gaza (May 14, 2025)
- The Swiss lawyer behind the rise of ‘golden passports’ (May 14, 2025)
- Switzerland Today: The Swiss government takes two important EU-related decisions, and former Credit Suisse executives will finally receive their bonuses (May 14, 2025)
- No house generation: the impossibility of buying property in Switzerland (May 12, 2025)
- Online quarrel reveals Swiss life of luxury of Cameroon’s ruling family (May 13, 2025)
- Toxic masculinity and the rollback of women’s rights (May 13, 2025)

Cost of the agent:  0.0060992


#### So now we have an agent that can answer questions using the LLM and also search the web for us.
#### Let's see what the agents does behind the scene.

In [14]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you

In [15]:
# Let's print the system prompt
print("""""", agent.system_prompt)

 You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to some tools.

The tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".
This Action/Observation can repeat N times, you should take several steps when needed.

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Action:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}

To provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "name": "final_answer

## Tools

A tool is an atomic function to be used by an agent. To be used by an LLM, it also needs a few attributes that constitute its API and will be used to describe to the LLM how to call this tool:

    A name
    A description
    Input types and descriptions
    An output type

The library provide a list of default tools: https://github.com/huggingface/smolagents/blob/28cfef22389a2830176b48be9fcc3e3d5793b87b/src/smolagents/default_tools.py#L102

- PythonInterpreterTool
- FinalAnswerTool
- UserInputTool
- DuckDuckGoSearchTool
- GoogleSearchTool
- VisitWebpageTool

With the `smolagents` library, there are two ways of declaring tool. Using the `@tool` decorator or using the `Tool` class.

The `@tool` decorator is a more concise way of declaring a tool, but it is less flexible than the `Tool` class.

###  Defining a Tool as a Python Class

In this class, we define:

- `name`: The tool’s name.
- `description`: A description used to populate the agent’s system prompt.
- `inputs`: A dictionary with keys type and description, providing information to help the Python interpreter process inputs.
- `output_type`: Specifies the expected output type.
- `forward`: The method containing the inference logic to execute.

In [16]:
from smolagents import Tool


class Sum(Tool):
    name = "sum"
    description = "This is a tool that can add two numbers. It returns the sum of the two numbers."
    inputs = {
        "number_1": {"type": "number", "description": "The first number to add."},
        "number_2": {"type": "number", "description": "The second number to add."},
    }
    output_type = "number"

    def forward(self, number_1: float, number_2: float) -> float:
        return number_1 + number_2

In [17]:
sum_tool = Sum()
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
)
agent.visualize()

ToolCallingAgent | gpt-4.1-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum          │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [18]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ sum 3 4                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum' with arguments: {'number_1': 3, 'number_2': 4}                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 7

[Step 1: Duration 21.72 seconds| Input tokens: 973 | Output tokens: 21]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '7'}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 7

[Step 2: Duration 1.15 seconds| Input tokens: 2,044 | Output tokens: 36]

7

Cost of the agent:  0.0008752


### The @tool Decorator

Using this approach, we define a function with:

- **A clear and descriptive function name** that helps the LLM understand its purpose.
- **Type hints for both inputs and outputs** to ensure proper usage.
- **A detailed description**, including an Args: section where each argument is explicitly described. These descriptions provide valuable context for the LLM, so it’s important to write them carefully.

In [19]:
from smolagents import tool


@tool
def sum_tool(number_1: float, number_2: float) -> float:
    """
    This is a tool that can add two numbers. It returns the sum of the two numbers.

    Args:
        number_1: The first number to add.
        number_2: The second number to add.
    """
    return number_1 + number_2

In [20]:
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
)
agent.visualize()

ToolCallingAgent | gpt-4.1-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [21]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ sum 3 4                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum_tool' with arguments: {'number_1': 3, 'number_2': 4}                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 7

[Step 1: Duration 1.33 seconds| Input tokens: 975 | Output tokens: 23]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '7'}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 7

[Step 2: Duration 1.50 seconds| Input tokens: 2,048 | Output tokens: 37]

7

Cost of the agent:  0.0008784000000000001


## Code Agent vs Tool Calling Agent

In [22]:
from datetime import date

today = date.today()
current_date = today.strftime("%Y-%m-%d")

In [23]:
from smolagents import DuckDuckGoSearchTool

web_search = DuckDuckGoSearchTool()
agent = ToolCallingAgent(
    model=model,
    tools=[sum_tool, web_search],
    verbosity_level=1,
    max_steps=10,
)
agent.visualize()

ToolCallingAgent | gpt-4.1-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
    │              │ your query (think a Google search) then       │ perform.                                     │
    │              │ returns the top search results.               │                                              │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [24]:
output = agent.run(
    f"You are an agent that can study financial market. Today's date is {current_date}. "
    + "What is the gain that Nvidia stock made in the last week?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent, model.model_id))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 2025-05-15. What is the gain that Nvidia      │
│ stock made in the last week?                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': 'Nvidia stock price 2025-05-08'}                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Error executing tool 'web_search' with arguments {"query": "Nvidia stock price 2025-05-08"}: 
DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 202 Ratelimit
Please try again or use another tool

[Step 1: Duration 2.77 seconds| Input tokens: 1,110 | Output tokens: 24]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The gain that Nvidia stock made in the last week is    │
│ $16.39.'}                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The gain that Nvidia stock made in the last week is $16.39.

[Step 4: Duration 1.28 seconds| Input tokens: 7,591 | Output tokens: 103]

The gain that Nvidia stock made in the last week is $16.39.

Cost of the agent:  0.0032012000000000004


In [25]:
from smolagents import CodeAgent

agent = CodeAgent(
    model=model,
    add_base_tools=False,
    tools=[sum_tool, web_search],
    verbosity_level=1,
    max_steps=10,
)
agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
    │              │ your query (think a Google search) then       │ perform.                                     │
    │              │ returns the top search results.               │                                              │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [26]:
output = agent.run(
    f"You are an agent that can study financial market. Today's date is {current_date}. "
    + "What is the gain that Nvidia stock made in the last week?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent, model.model_id))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 2025-05-15. What is the gain that Nvidia      │
│ stock made in the last week?                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for Nvidia stock price on 2025-05-08                                                                    
  price_week_ago = web_search(query="Nvidia stock price May 8, 2025")                                              
  print("Nvidia stock price on 2025-05-08:", price_week_ago)                                                       
                                                                                                                   
  # Search for Nvidia stock price on 2025-05-15                                                                    
  price_today = web_search(query="Nvidia stock price May 15, 2025")                                                
  print("Nvidia stock price on 2025-05-15:", price_today)                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Nvidia stock price on 2025-05-08: ## Search Results

[Nvidia Stock Price In May 2025 | StatMuse 
Money](https://www.statmuse.com/money/ask/nvidia-stock-price-in-may-2025)
The closing price for NVIDIA (NVDA) this month is $129.93, yesterday. It is up 14.9% for the month. The latest 
price is $129.83.

[Nvidia Stock Price Prediction 2025, 2026, 2027-2029](https://longforecast.com/nvidia)
Nvidia stock closed at 129.93 dollars the previous day. Nvidia Share Price Prediction For Next Months and Years 
Nvidia stock price forecast for May 2025. The forecast for beginning 112 dollars. Maximum price 165, minimum 111. 
Averaged Nvidia stock price for the month 135. At the end 150 dollars, change for May 33.9%. Nvidia stock 
prediction for ...

[Prediction: Nvidia Stock Is Going to Soar After May 
28](https://www.fool.com/investing/2025/05/13/prediction-nvidia-stock-is-going-soar-after-may-28/)
Based on the company's fiscal 2025 EPS of $2.99, its stock trades at a price-to-earnings (P/E) ratio of nearly 40 
as of this writing, which is a steep discount to its 10-year average of 59.7.

[NVIDIA (NASDAQ: NVDA) Price Prediction and Forecast 2025-2030 for May 
6](https://247wallst.com/forecasts/2025/05/06/nvidia-nvda-price-prediction-and-forecast/)
NVIDIA continues to fuel the AI stock rally in 2024. Our analysis shows that may be the case for the foreseeable 
future.

[Historical NVDA stock prices (quote) - NVIDIA Corporation](https://stockinvest.us/stock-price/NVDA)
Historical stock prices refer to a stock's recorded prices at various past points. These prices include several key
figures that help investors and analysts evaluate a stock's performance over time: Open: Open price for the trading
day. High: Highest price for the trading day. Low: Lowest price for the trading day. Close: Close price for the 
trading day. Additionally, historical prices ...

[NVIDIA STOCK PRICE FORECAST TOMORROW, WEEK, 2025, 2026, 2027 - 30 Rates](https://30rates.com/nvidia)
Nvidia stock price prediction for tomorrow, near days, this week and this month. Short term NVDA stock forecast 
updated TODAY! Nvidia share price prediction for 2025, 2026, 2027 in the table.

[Nvidia Stock Price Forecast 2025 to 2050 based on May 
2025](https://futurevaluejournal.com/nvidia-stock-price-forecast/)
Discover the latest Nvidia stock price forecast, including expert analysis and insights on market trends, growth 
potential & future performance.

[NVIDIA (NVDA) Historical Stock Price Data](https://stockanalysis.com/stocks/nvda/history/)
A complete stock price history for NVIDIA Corporation (NVDA) stock, starting from its first trading day. Includes 
open, high, low, close and volume.

[Nvidia Stock Price Prediction: 2025, 2026, 2030 - 
Benzinga](https://www.benzinga.com/money/nvidia-stock-price-prediction)
Explore expert predictions for Nvidia's stock price in 2025, 2026 and 2030, with insights on market trends, AI 
growth and future valuations.

[Stock market today: S&P 500 wipes out 2025 losses as Nvidia powers tech 
...](https://finance.yahoo.com/news/live/stock-market-today-sp-500-wipes-out-2025-losses-as-nvidia-powers-tech-rall
y-133029337.html)
The euphoria over the US-China trade extended for a second day as a tech rally led stocks higher on Tuesday.
Nvidia stock price on 2025-05-15: ## Search Results

[Nvidia Stock Price In May 2025 | StatMuse 
Money](https://www.statmuse.com/money/ask/nvidia-stock-price-in-may-2025)
The closing price for NVIDIA (NVDA) this month is $129.93, yesterday. It is up 14.9% for the month. The latest 
price is $129.83.

[Nvidia Stock Price Prediction 2025, 2026, 2027-2029](https://longforecast.com/nvidia)
Averaged Nvidia stock price for the month 135. At the end 150 dollars, change for May 33.9%. Nvidia stock 
prediction for June 2025. The forecast for beginning 150 dollars. Maximum price 187, minimum 137. Averaged Nvidia 
stock price for the month 162. At the end 173 dollars, change for June 15.3%. Nvidia stock price forecast fo

[Step 1: Duration 6.71 seconds| Input tokens: 2,082 | Output tokens: 191]

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Closing prices retrieved from search results                                                                   
  closing_price_may_8 = 117.37                                                                                     
  closing_price_may_15 = 129.93                                                                                    
                                                                                                                   
  # Calculate weekly gain in percent                                                                               
  weekly_gain = ((closing_price_may_15 - closing_price_may_8) / closing_price_may_8) * 100                         
                                                                                                                   
  final_answer(f"Nvidia stock gain over the last week is {weekly_gain:.2f}%.")                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Nvidia stock gain over the last week is 10.70%.

[Step 4: Duration 4.50 seconds| Input tokens: 19,132 | Output tokens: 750]

Nvidia stock gain over the last week is 10.70%.

Cost of the agent:  0.008852800000000001


## Agent Hierarchy / MultiAgent

It is also possible to use a multi-agent system, where multiple agents can be used to solve a problem. This is useful when the problem is too complex for a single agent to solve. In this case, the agents can communicate with each other to solve the problem.

Another advantage is context size, as a single agent will store the full history of the steps, while a multi-agent system will store only the history of the steps of the agent that is currently active.

In [27]:
from smolagents import CodeAgent, DuckDuckGoSearchTool


web_agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
    add_base_tools=False,
    name="information_retriever_agent",
    description="An agent that can be called to run web search to obtain information. Call it as a function using the **task** argument.",
    verbosity_level=1,
)

manager_agent = CodeAgent(
    tools=[sum_tool],
    model=model,
    managed_agents=[web_agent],
    verbosity_level=1,
    description="An agent that manages other agent.",
    max_steps=10,
)

manager_agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
│   │              │ returns the sum of the two numbers.           │ add.                                         │
│   │              │                                               │ number_2 (`number`): The second number to    │
│   │              │                                               │ add.                                         │
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    └── information_retriever_agent | CodeAgent | gpt-4.1-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: An agent that can be called to run web search to obtain information. Call it as a 
        │   function using the **task** argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name         ┃ Description                               ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search   │ Performs a duckduckgo web search based on │ query (`string`): The search query to    │
            │              │ your query (think a Google search) then   │ perform.                                 │
            │              │ returns the top search results.           │                                          │
            │ final_answer │ Provides a final answer to the given      │ answer (`any`): The final answer to the  │
            │              │ problem.                                  │ problem                                  │
            └──────────────┴───────────────────────────────────────────┴──────────────────────────────────────────┘

In [28]:
output = manager_agent.run(
    f"You are an agent that can study financial market. Today's date is {current_date}. "
    + "What is the gain that Nvidia stock made in the last week?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(manager_agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 2025-05-15. What is the gain that Nvidia      │
│ stock made in the last week?                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  task = (                                                                                                         
      "Retrieve the closing stock price of Nvidia (ticker symbol NVDA) on 2025-05-15 and 2025-05-08. "             
      "Please provide the prices in a clear format, specifying both dates and closing prices."                     
  )                                                                                                                
  nvidia_prices = information_retriever_agent(task=task)                                                           
  print(nvidia_prices)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭───────────────────────────────────── New run - information_retriever_agent ─────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'information_retriever_agent'.                                                     │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Retrieve the closing stock price of Nvidia (ticker symbol NVDA) on 2025-05-15 and 2025-05-08. Please provide    │
│ the prices in a clear format, specifying both dates and closing prices.                                         │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  price_start = 111  # Approximate forecasted low price on May 8, 2025                                             
  price_end = 165    # Approximate forecasted high price on May 15, 2025                                           
                                                                                                                   
  gain_percent = ((price_end - price_start) / price_start) * 100                                                   
  final_answer(f"Estimated gain for Nvidia stock in the last week (May 8 to May 15, 2025) is approximately         
  {gain_percent:.2f}% based on forecasted price ranges.")                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Estimated gain for Nvidia stock in the last week (May 8 to May 15, 2025) is approximately 
48.65% based on forecasted price ranges.

[Step 2: Duration 5.31 seconds| Input tokens: 7,982 | Output tokens: 717]

Estimated gain for Nvidia stock in the last week (May 8 to May 15, 2025) is approximately 48.65% based on forecasted price ranges.

Cost of the agent:  0.00434


## Visualize Your Agent

In [ ]:
from smolagents import (
    GradioUI,
)

web_agent = CodeAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
    name="information_retriever_agent",
    description="An agent that can be called to run web search to obtain information. Call it as a function using the **task** argument.",
    verbosity_level=0,
)

manager_agent = CodeAgent(
    name="ManagerAgent",
    tools=[],
    model=model,
    managed_agents=[web_agent],
    verbosity_level=0,
    description="An agent that manages other agent.",
    max_steps=10,
)

GradioUI(manager_agent).launch(
    share=True
)  # This is necessary to make it work in Renku but be careful

# !!! Do not forget to stop the process of the previous cell before executing the next ones !!! 

## Another Multi Agent Example

We will create:

- an agent that can read a document and summarize it

- an agent that can search the web

- a manager agent that handle both previous agents

In [29]:
import requests
from helpers.data_processing import SimpleChunker, PDFExtractor


@tool
def read_pdf_tool(pdf_file_path: str) -> str:
    """
    This tool reads a PDF file and returns the text content of the PDF file.

    Args:
        pdf_file_path: The path to the PDF file.
    """
    response, text, images = PDFExtractor().extract_text_and_images(pdf_file_path)
    return text

In [30]:
tmp = read_pdf_tool("../../data/hypoxy_stat_1page.pdf")
tmp

'Please cite this article in press as: Blume et al., HypoxyStat, a small-molecule form of hypoxia therapy that increases oxygen-hemoglobin affinity, Cell (2025), https://doi.org/10.1016/j.cell.2025.01.029\n\n![](_page_0_Picture_2.jpeg)\n\n### Article HypoxyStat, a small-molecule form of hypoxia therapy that increases oxygen-hemoglobin affinity\n\nSkyler Y. Blume,[1,](#page-0-0)[2](#page-0-1) Ankur Garg,[1,](#page-0-0)[2](#page-0-1) Yolanda Martı´-Mateos,[1,](#page-0-0)[2](#page-0-1) Ayush D. Midha,[1,](#page-0-0)[2](#page-0-1) Brandon T. L. Chew,[1,](#page-0-0)[2](#page-0-1) Baiwei Lin,[3](#page-0-2) Cecile Yu,[3](#page-0-2) Ryan Dick,[3](#page-0-2) Patrick S. Lee,[3](#page-0-2) Eva Situ,[3](#page-0-2) Richa Sarwaikar,[3](#page-0-2) Eric Green,[3](#page-0-2) Vyas Ramanan,[3](#page-0-2) Gijsbert Grotenbreg,[3](#page-0-2) Maarten Hoek,[3](#page-0-2) Christopher Sinz,[3](#page-0-2) and Isha H. Jain[1,](#page-0-0)[2,](#page-0-1)[4,](#page-0-3)[5,](#page-0-4)[\\\\*](#page-0-5)\n\n<span id="

In [31]:
agent_read_pdf = CodeAgent(
    name="read_pdf_agent",
    description="Reads and summarize a PDF file. Call it as a function using the **task** argument.",
    tools=[read_pdf_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

agent_web_search = CodeAgent(
    name="web_search_agent",
    description="Runs web searches for you. Call it as a function using the **task** argument.",
    tools=[DuckDuckGoSearchTool(), visit_webpage_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

agent = CodeAgent(
    name="medical_agent",
    tools=[],
    model=model,
    add_base_tools=False,
    managed_agents=[agent_read_pdf, agent_web_search],
    verbosity_level=1,
)

agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    ├── read_pdf_agent | CodeAgent | gpt-4.1-mini
    │   ├── ✅ Authorized imports: []
    │   ├── 📝 Description: Reads and summarize a PDF file. Call it as a function using the **task** argument.
    │   └── 🛠️ Tools:
    │       ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    │       ┃ Name          ┃ Description                              ┃ Arguments                                ┃
    │       ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │       │ read_pdf_tool │ This tool reads a PDF file and returns   │ pdf_file_path (`string`): The path to    │
    │       │               │ the text content of the PDF file.        │ the PDF file.                            │
    │       │ web_search    │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
    │       │               │ on your query (think a Google search)    │ perform.                                 │
    │       │               │ then returns the top search results.     │                                          │
    │       │ visit_webpage │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
    │       │               │ reads its content as a markdown string.  │ to visit.                                │
    │       │               │ Use this to browse webpages.             │                                          │
    │       │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to the  │
    │       │               │ problem.                                 │ problem                                  │
    │       └───────────────┴──────────────────────────────────────────┴──────────────────────────────────────────┘
    └── web_search_agent | CodeAgent | gpt-4.1-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Runs web searches for you. Call it as a function using the **task** argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description                              ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search    │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
            │               │ on your query (think a Google search)    │ perform.                                 │
            │               │ then returns the top search results.     │                                          │
            │ visit_webpage │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
            │               │ reads its content as a markdown string.  │ to visit.                                │
            │               │ Use this to browse webpages.             │                                          │
            │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to th

In [32]:
output = agent.run(
    "Your task is the following: "
    + "Can you read the PDF file at '../../data/hypoxy_stat_1page.pdf' and tell me what it is about? Also, can you give me the wikipedia definition of the area of research?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────── New run - medical_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ Your task is the following: Can you read the PDF file at '../../data/hypoxy_stat_1page.pdf' and tell me what it │
│ is about? Also, can you give me the wikipedia definition of the area of research?                               │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  pdf_summary = read_pdf_agent(task="../../data/hypoxy_stat_1page.pdf")                                            
  print(pdf_summary)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────────────────────────────────────── New run - read_pdf_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'read_pdf_agent'.                                                                  │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ ../../data/hypoxy_stat_1page.pdf                                                                                │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "pdf_summary": "HypoxyStat is a novel small-molecule therapeutic that increases hemoglobin's affinity for    
  oxygen, thereby inducing systemic hypoxia to replicate the benefits of chronic inhaled hypoxia therapy. It has   
  demonstrated promising effects in mouse models of mitochondrial diseases by prolonging lifespan and improving    
  disease symptoms, offering a more practical alternative to inhaled hypoxia.",                                    
      "wikipedia_definition": "Hypoxia therapy involves controlled exposure to low oxygen levels (hypoxia) and     
  explores its potential benefits in diseases including mitochondrial diseases. It aims to induce adaptive         
  cellular responses to improve mitochondrial function, reduce oxidative stress, and enhance cellular survival.    
  Systemic hypoxia refers to a reduced oxygen supply throughout the body, either pathologically or                 
  therapeutically induced, and is researched for its potential to slow mitochondrial disease progression. This     
  area of research focuses on manipulating oxygen availability to elicit beneficial cellular adaptations in        
  mitochondrial disorders."                                                                                        
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'pdf_summary': "HypoxyStat is a novel small-molecule therapeutic that increases hemoglobin's 
affinity for oxygen, thereby inducing systemic hypoxia to replicate the benefits of chronic inhaled hypoxia 
therapy. It has demonstrated promising effects in mouse models of mitochondrial diseases by prolonging lifespan and
improving disease symptoms, offering a more practical alternative to inhaled hypoxia.", 'wikipedia_definition': 
'Hypoxia therapy involves controlled exposure to low oxygen levels (hypoxia) and explores its potential benefits in
diseases including mitochondrial diseases. It aims to induce adaptive cellular responses to improve mitochondrial 
function, reduce oxidative stress, and enhance cellular survival. Systemic hypoxia refers to a reduced oxygen 
supply throughout the body, either pathologically or therapeutically induced, and is researched for its potential 
to slow mitochondrial disease progression. This area of research focuses on manipulating oxygen availability to 
elicit beneficial cellular adaptations in mitochondrial disorders.'}

[Step 3: Duration 6.70 seconds| Input tokens: 18,659 | Output tokens: 1,685]

Cost of the agent:  0.010159600000000001


## Exercise 1: Create your own RAG Agent

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/beating_gaia/classical_vs_agentic_rag.png)

First we will create a simple agent that can answer questions on a knowledge base, AKA a **RAG agent**.

1. Define a tool that retrieve documents from a knowledge base
2. Define an agent that uses the tool to retrieve documents and answer questions

In [33]:
from helpers.constants_and_data_classes import Chunk
from helpers.data_processing import SimpleChunker, PDFExtractorAPI
from helpers.embedding import (
    OpenAITextEmbeddings,
    compute_openai_large_embedding_cost,
)
from helpers.vectorstore import (
    ChromaDBVectorStore,
    VectorStoreRetriever,
)

### Retriever Pipeline

In [34]:
data_folder = "../../data"

pdf_files = [
    "Explainable_machine_learning_prediction_of_edema_a.pdf",
    "Modeling tumor size dynamics based on real‐world electronic health records.pdf",
]
example_pdf_file = "Explainable_machine_learning_prediction_of_edema_a.pdf"
example_pdf_path = os.path.join(data_folder, example_pdf_file)

vector_store_collection = "text_collection"

In [35]:
data_extractor = PDFExtractor()
_, text, _ = data_extractor.extract_text_and_images(example_pdf_path)

In [36]:
file_metadata = {"source_text": example_pdf_file}

text_chunker = SimpleChunker(max_chunk_size=1000)

chunks = text_chunker.chunk_text(text, file_metadata)

In [37]:
embedding_model = OpenAITextEmbeddings()
embeddings = embedding_model.get_embedding([chunk.content for chunk in chunks])

In [38]:
vector_store = ChromaDBVectorStore(vector_store_collection)
vector_store.insert_chunks(chunks, embeddings)

In [39]:
retriever = VectorStoreRetriever(embedding_model, vector_store)
results = retriever.retrieve("Who are the authors of the paper=", 5)
results

[[{'chunk_id': '48',
   'score': 1.266050100326538,
   'chunk': Chunk(chunk_id=48, content='performed the research. F.A. and N.T. analyzed the data. F.A., N.T., and R.C. contributed new reagents/analytical tools. ### **FUNDING INFORMATION** This study was sponsored by the healthcare business of Merck KGaA, Darmstadt, Germany (CrossRef Funder ID:10.13039/100009945). Editorial support was provided by Syneos Health, London, UK, and funded by the healthcare business of Merck KGaA, Darmstadt, Germany. ### **CONFLICT OF INTEREST STATEMENT** N.T. is an employee of Ares Trading S.A., Lausanne, Switzerland, an affiliate of Merck KGaA, Darmstadt, Germany. R.S., R.B. and A.J. are employees of the healthcare business of Merck KGaA, Darmstadt, Germany. K.B. was an employee of the healthcare business of Merck KGaA, Darmstadt, Germany at the time of the study. V.G. is an employee of Ares Trading S.A. Eysins, Switzerland, an affiliate of Merck KGaA, Darmstadt, Germany. F.A. and R.C.', metadata={'sourc

### Toolify

In [ ]:
# Create the retriever tool
@tool
def retriever_tool(query: str, number_of_chunks: int) -> list:
    ## Fill the docstring here

    return retriever.retrieve(query, number_of_chunks)


# Create the CodeAgent with the tool
rag_agent = CodeAgent(tools=[retriever_tool], model=model)
rag_agent.visualize()

In [40]:
# Create the retriever tool
@tool
def retriever_tool(query: str, number_of_chunks: int) -> list:
    """
    This is a tool that can search a document and extract the related information based on the query given. It returns a list of string
    Only use this tool if necessary to answer the query otherwise rely on your internal knowledge.

    Args:
        query: The user query
        number_of_chunks: number of chunks to return, by default it's 5.
    """

    return retriever.retrieve(query, number_of_chunks)


# Create the CodeAgent with the tool
rag_agent = CodeAgent(tools=[retriever_tool], model=model)
rag_agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name           ┃ Description                                  ┃ Arguments                                   ┃
    ┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ retriever_tool │ This is a tool that can search a document    │ query (`string`): The user query            │
    │                │ and extract the related information based on │ number_of_chunks (`integer`): number of     │
    │                │ the query given. It returns a list of string │ chunks to return, by default it's 5.        │
    │                │ Only use this tool if necessary to answer    │                                             │
    │                │ the query otherwise rely on your internal    │                                             │
    │                │ knowledge.                                   │                                             │
    │ final_answer   │ Provides a final answer to the given         │ answer (`any`): The final answer to the     │
    │                │ problem.                                     │ problem                                     │
    └────────────────┴──────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [41]:
# Call the agent
output = rag_agent.run(
    "According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade 2+)?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade     │
│ 2+)?                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  results = retriever_tool(query="SHAP analysis influential factors predicting higher-grade edema Grade 2+",       
  number_of_chunks=5)                                                                                              
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
[[{'chunk_id': '39', 'score': 0.5653574466705322, 'chunk': Chunk(chunk_id=39, content='Points are colored based on 
the edema grade at the following safety visit. SHAP, Shapley Additive exPlanations. with higher grades of edema, 
particularly grade 2+. On the other hand, for higher albumin levels the corresponding SHAP values are mostly 
negative and ranging from 0 to −0.5, suggesting a reduced risk of edema of grade 2+. The association between age 
greater than 70years and an increased likelihood of edemas of grades 2+ was also confirmed. Additionally, for all 
ages, higher SHAP values were assigned to patients who experienced edemas, particularly of grade 2+. Finally, 
within low ranges of cumulated dose in the interval [ *t* − 14 days, *t* ] normalized over 14days, higher SHAP 
values were assigned to samples corresponding to edemas of grades 2+. This could reflect the tendency to adjust 
administered doses in those cases where the risk of edema was identified. # **DISCUSSION**', 
metadata={'source_text': 'Explainable_machine_learning_prediction_of_edema_a.pdf', 'document_chunk_id': 39}, 
data_type=<DataType.TEXT: 'text'>, score=None)}, {'chunk_id': '42', 'score': 0.7433754205703735, 'chunk': 
Chunk(chunk_id=42, content='The presence of such input is ensuring high model performances when predicting 
persistence of a given edema grade. The sensitivity analysis to the inclusion of this input revealed a decline in 
the mean cross-validation F1 score of ~0.350 when it was removed as a candidate predictor. However, the use of 
Isotonic Regressions ensures the estimations of correctly calibrated probabilities. For instance, while low 
predicted probabilities, for example, for edemas of grade 2+, might lead to a classification error, they can still 
provide valuable details on existing risk of occurrence of an adverse event (Figure [S5\\)](#page-11-0). Moreover, 
sensitivity analysis together with the SHAP importance highlighted interesting patterns with respect to the 
exposure-related features. Referencing Figure [3](#page-7-1), lower values of the dose [ *t* − 14 days, *t* ] are 
associated with a decreased probability of edema of grades 2+ at subsequent visits.', metadata={'source_text': 
'Explainable_machine_learning_prediction_of_edema_a.pdf', 'document_chunk_id': 42}, data_type=<DataType.TEXT: 
'text'>, score=None)}, {'chunk_id': '37', 'score': 0.7551815509796143, 'chunk': Chunk(chunk_id=37, 
content='![](_page_7_Figure_0.jpeg) <span id="page-7-0"></span>**FIGURE 2** Global input importance via mean SHAP 
values. Ranking of the model input for the most influential to the less influential for the model. The y-axis 
indicates the average change in the predicted probability of edema by grade, on average across the entire test set.
SHAP, Shapley Additive exPlanations. <span id="page-7-1"></span>**FIGURE 3** SHAP values – contribution of the 
inputs toward the predicted probabilities of edemas of grade 2+. List of the eight most influential inputs with 
respect to the predicted probabilities of edemas of grades 2+. Each point on the plot is a SHAP value for a 
covariate at a specific patient visit. The position on the y-axis indicates the covariate importance and on the 
x-axis the impact on the predicted probability. Color represents the value of the covariate. SHAP, Shapley Additive
exPlanations. with an increase in the predicted probability of edemas of grades 2+.', metadata={'source_text': 
'Explainable_machine_learning_prediction_of_edema_a.pdf', 'document_chunk_id': 37}, data_type=<DataType.TEXT: 
'text'>, score=None)}, {'chunk_id': '32', 'score': 0.7893579006195068, 'chunk': Chunk(chunk_id=32, 
content='Consistently with the above sensitivity analysis, past current edema grade was found to be the most 
influential input, particularly if a same grade persisted to the following safety visit. The exposure-derived 
features were also informative for the model probability predictions. Albumin was found as the m

[Step 1: Duration 3.21 seconds| Input tokens: 2,046 | Output tokens: 112]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  influential_factors_summary = (                                                                                  
      "According to SHAP analysis, the most influential factors for predicting higher-grade edema (Grade 2+)       
  include: "                                                                                                       
      "1) the current edema grade, particularly if a grade 2+ persisted to the following visit, "                  
      "2) lower albumin levels, which are associated with increased risk, "                                        
      "3) older age, especially above 70 years, which correlates with higher likelihood of grade 2+ edema, "       
      "4) and cumulative dose over the 14 days prior to prediction, with lower doses associated with decreased     
  risk."                                                                                                           
  )                                                                                                                
  final_answer(influential_factors_summary)                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: According to SHAP analysis, the most influential factors for predicting higher-grade edema 
(Grade 2+) include: 1) the current edema grade, particularly if a grade 2+ persisted to the following visit, 2) 
lower albumin levels, which are associated with increased risk, 3) older age, especially above 70 years, which 
correlates with higher likelihood of grade 2+ edema, 4) and cumulative dose over the 14 days prior to prediction, 
with lower doses associated with decreased risk.

[Step 2: Duration 6.29 seconds| Input tokens: 5,679 | Output tokens: 362]

In [44]:
output = rag_agent.run("What is the highest court of the USA?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the highest court of the USA?                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The highest court of the USA is the Supreme Court of the United States.")                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The highest court of the USA is the Supreme Court of the United States.

[Step 1: Duration 1.59 seconds| Input tokens: 2,032 | Output tokens: 66]

## Exercise 2: Make it a multi-agent system

Now that we built a RAG agent, we will improve it by adding a web search tool to it. This way, if the agent can't find the answer in the knowledge base, it will search the web for it.

However, we will transform our agent into a multi-agent system. This way, we will have one agent responsible for answering questions using the knowledge base and another agent responsible for searching the web.

1. Define a tool that searches the web
2. Define a new agent that uses the web search tool
3. Create a multi-agent system that uses both agents


In [45]:
multi_rag_agent = ...

In [46]:


#
from smolagents import DuckDuckGoSearchTool

web_search_agent = CodeAgent(
    name="web_search_agent",
    description="Runs web searches for you. Call it as a function using the **task** argument.",
    tools=[DuckDuckGoSearchTool(), visit_webpage_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

rag_agent = CodeAgent(
    name="medical_literature_agent",
    description="Retrieve information from medical litterature. Call it as a function using the **task** argument.",
    tools=[retriever_tool],
    model=model,
)

multi_rag_agent = CodeAgent(
    tools=[],
    model=model,
    add_base_tools=False,
    managed_agents=[rag_agent, web_search_agent],
    verbosity_level=1,
)



In [47]:
# Call the agent
output = multi_rag_agent.run(
    "According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade 2+)?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade     │
│ 2+)?                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_query = ("SHAP analysis factors most influential in predicting higher-grade edema Grade 2 or higher")     
  web_search_results = web_search_agent(search_query)                                                              
  print(web_search_results)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭────────────────────────────────────────── New run - web_search_agent ───────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'web_search_agent'.                                                                │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ SHAP analysis factors most influential in predicting higher-grade edema Grade 2 or higher                       │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 1: Duration 35.61 seconds| Input tokens: 4,789 | Output tokens: 564]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The most influential factors in predicting higher-grade edema (Grade 2+) according to SHAP         
  analysis are lower albumin levels (hypoalbuminemia), older age particularly above 70 years, and multiple         
  time-varying clinical covariates including comorbidities, biomarkers, medications, vital signs, and prior edema  
  observations. Higher albumin levels tend to reduce the risk, while advanced age increases the predicted          
  likelihood of severe edema.")                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The most influential factors in predicting higher-grade edema (Grade 2+) according to SHAP 
analysis are lower albumin levels (hypoalbuminemia), older age particularly above 70 years, and multiple 
time-varying clinical covariates including comorbidities, biomarkers, medications, vital signs, and prior edema 
observations. Higher albumin levels tend to reduce the risk, while advanced age increases the predicted likelihood 
of severe edema.

[Step 2: Duration 3.45 seconds| Input tokens: 7,688 | Output tokens: 782]

In [48]:
output = multi_rag_agent.run("What is the current price of Nvidia stock?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the current price of Nvidia stock?                                                                      │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  nvidia_stock_price_info = web_search_agent(task="current Nvidia stock price")                                    
  print(nvidia_stock_price_info)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭────────────────────────────────────────── New run - web_search_agent ───────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'web_search_agent'.                                                                │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ current Nvidia stock price                                                                                      │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 1: Duration 44.45 seconds| Input tokens: 5,179 | Output tokens: 352]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The current Nvidia stock price is approximately $116.39.")                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The current Nvidia stock price is approximately $116.39.

[Step 2: Duration 1.44 seconds| Input tokens: 7,678 | Output tokens: 410]